### Additional Assets

- 삼성전자 (005930)
- AAPL
- MSFT
- TSLA
- 미국채 10년물
- 달러 / 엔
- 달러 / 원
- WTI Crude

In [3]:
from utils import *

In [5]:
import yfinance as yf

stock_tickers = ['AAPL','MSFT','TSLA']
stocks = yf.download(tickers = stock_tickers, start = start_date, end = end_date)['Adj Close']

[*********************100%%**********************]  3 of 3 completed


In [9]:
stocks_ret = stocks.pct_change()
cumret = ((1 + stocks_ret).cumprod() - 1).fillna(0)

In [10]:
cumret

Ticker,AAPL,MSFT,TSLA
Date,,,
2014-01-02,0.000000,0.000000,0.000000
2014-01-03,-0.021966,-0.006728,-0.003598
2014-01-06,-0.016632,-0.027718,-0.020653
2014-01-07,-0.023665,-0.020183,-0.004930
2014-01-08,-0.017482,-0.037675,0.007861
...,...,...,...
2024-03-18,9.043560,12.361485,16.368421
2024-03-19,9.180003,12.492435,16.120586
2024-03-20,9.329742,12.614742,16.554297


In [11]:
days = (i for i in range(3, 62, 2))
threshold = tuple([-i * 0.01 for i in range(3, 21)])

In [12]:
items = [(i, j) for i in days for j in threshold]

In [18]:
asset = dict()

In [19]:
for i in cumret.columns :
    asset[i] = pd.DataFrame(index = stocks.index)

In [24]:
for i in cumret.columns :
    for j in range(1, 501) :
        asset[i]['-' + str(j) + 'days'] = (stocks[i] - stocks[i].shift(j)) / stocks[i]

In [27]:
for i in cumret.columns :
    asset[i].to_csv(i + '_returns_data_adj.csv')
    asset[i].to_excel(i + '_returns_data_adj.xlsx')

In [28]:
for i in cumret.columns :
    asset[i].index = range(len(stocks))

In [32]:
cumret

Ticker,AAPL,MSFT,TSLA
Date,,,
2014-01-02,0.000000,0.000000,0.000000
2014-01-03,-0.021966,-0.006728,-0.003598
2014-01-06,-0.016632,-0.027718,-0.020653
2014-01-07,-0.023665,-0.020183,-0.004930
2014-01-08,-0.017482,-0.037675,0.007861
...,...,...,...
2024-03-18,9.043560,12.361485,16.368421
2024-03-19,9.180003,12.492435,16.120586
2024-03-20,9.329742,12.614742,16.554297


In [34]:
asset

{'AAPL':         -1days    -2days    -3days    -4days    -5days    -6days    -7days  \
 0          NaN       NaN       NaN       NaN       NaN       NaN       NaN   
 1    -0.022459       NaN       NaN       NaN       NaN       NaN       NaN   
 2     0.005424 -0.016914       NaN       NaN       NaN       NaN       NaN   
 3    -0.007203 -0.001740 -0.024239       NaN       NaN       NaN       NaN   
 4     0.006293 -0.000865  0.004563 -0.017793       NaN       NaN       NaN   
 ...        ...       ...       ...       ...       ...       ...       ...   
 2568  0.006332  0.004145  0.014909  0.002821  0.005584  0.017212  0.027170   
 2569  0.013403  0.019650  0.017492  0.028112  0.016186  0.018912  0.030384   
 2570  0.014496  0.027705  0.033861  0.031734  0.042201  0.030447  0.033134   
 2571 -0.042598 -0.027484 -0.013713 -0.007294 -0.009512  0.001400 -0.010854   
 2572  0.005282 -0.037091 -0.022057 -0.008359 -0.001974 -0.004179  0.006675   
 
         -8days    -9days   -10days  ...  

In [61]:
threshold

(-0.03,
 -0.04,
 -0.05,
 -0.06,
 -0.07,
 -0.08,
 -0.09,
 -0.1,
 -0.11,
 -0.12,
 -0.13,
 -0.14,
 -0.15,
 -0.16,
 -0.17,
 -0.18,
 -0.19,
 -0.2)

In [59]:
asset['AAPL']['-1days'][asset['AAPL']['-1days'] < -0.03]

17     -0.086871
168    -0.044067
184    -0.039644
230    -0.033545
268    -0.036284
          ...   
2413   -0.050442
2435   -0.037122
2436   -0.030131
2516   -0.037115
2571   -0.042598
Name: -1days, Length: 111, dtype: float64

In [79]:
dataframes = dict()
dataframes_plot = dict()

for asset_index in ['AAPL','MSFT','TSLA'] :
    dataframes[asset_index] = dict()
    dataframes_plot[asset_index] = dict()
    for d, thres in tqdm(items) :
        thres = round(-thres * 100, 1)
        dataframes[asset_index][f'day{d}_{thres}per'] = pd.DataFrame(index = stocks.index)
        dataframes_plot[asset_index][f'day{d}_{thres}per_plot'] = pd.DataFrame(index = stocks.index)
        
        for i in range(1, 501) :
            try:
                temp = cumret[asset_index].iloc[
                            asset[asset_index][asset[asset_index][str(-i) + 'days'] < -thres * 0.01].index + d
                        ] - cumret[asset_index].iloc[asset[asset_index][asset[asset_index][str(-i) + 'days'] < -thres * 0.01].index].values
                
            except IndexError:
                print(f"IndexError occurred for {asset_index} at iteration {i}, skipping...")
                temp = pd.DataFrame(
                    index = stocks.index
                )
                temp['temp'] = np.nan
                
            dataframes[asset_index][f'day{d}_{thres}per'] = pd.concat(
                    [dataframes[asset_index][f'day{d}_{thres}per'], temp], axis = 1
            )
            dataframes_plot[asset_index][f'day{d}_{thres}per_plot'] = pd.concat(
                [dataframes_plot[asset_index][f'day{d}_{thres}per_plot'], temp], axis = 0
            )
        dataframes[asset_index][f'day{d}_{thres}per'].columns = [str(-i) + 'days' for i in range(1, 501)]
        dataframes[asset_index][f'day{d}_{thres}per'].to_excel(f'./excel/{asset_index}_day{d}_{int(thres)}per.xlsx')

  0%|          | 0/540 [00:00<?, ?it/s]

IndexError occurred for AAPL at iteration 1, skipping...
IndexError occurred for AAPL at iteration 2, skipping...
IndexError occurred for AAPL at iteration 14, skipping...
IndexError occurred for AAPL at iteration 15, skipping...
IndexError occurred for AAPL at iteration 16, skipping...
IndexError occurred for AAPL at iteration 17, skipping...
IndexError occurred for AAPL at iteration 18, skipping...
IndexError occurred for AAPL at iteration 19, skipping...
IndexError occurred for AAPL at iteration 20, skipping...
IndexError occurred for AAPL at iteration 21, skipping...
IndexError occurred for AAPL at iteration 22, skipping...
IndexError occurred for AAPL at iteration 23, skipping...
IndexError occurred for AAPL at iteration 24, skipping...
IndexError occurred for AAPL at iteration 25, skipping...
IndexError occurred for AAPL at iteration 26, skipping...
IndexError occurred for AAPL at iteration 27, skipping...
IndexError occurred for AAPL at iteration 28, skipping...
IndexError occur

  0%|          | 0/540 [00:03<?, ?it/s]


KeyboardInterrupt: 